In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

ModuleNotFoundError: No module named 'sklearn'

# 8 Morphological Descriptors

## Read data

In [ ]:
csv_path = "data/barley/extracted_phenotypes/area_morphology"

li = []
for i, file in enumerate([x for x in os.listdir(csv_path) if x.endswith(".csv")]):
    _df = pd.read_csv(os.path.join(csv_path,file), index_col=0)
    name = file.split(".")[0]
    _df.insert(loc=0,column="cultivar",value = name)
    _df.insert(loc=1,column="int_category",value = i)
    li.append(_df)

#create a dataframe from list
df1 = pd.concat(li, axis=0, ignore_index=True)

# retain only filtered values (i.e. removing non integral area such as awn containing, incomplete detection, partial appearance)
df1 = df1[df1["filter_level"] == 3]

# for easier interpretation of the dataframe
df1 = df1.sort_values('AS_seed_area')
df1.reset_index(drop=True, inplace=True)

In [ ]:
df1.head()

In [ ]:
#the variable that will be used in are from column 7 to 15
df1.iloc[:,7:15].describe()

In [ ]:
#define the color ordering in visualization
order = ['J647', 'C656', 'C346', 'N009', 'C319', 'K735', 'K692', 'J247', 'U353',
       'J064', 'T567', 'I622', 'I304', 'I335', 'U051', 'I626', 'B669', 'E245',
       'E612']

## Simple Visualization

In [ ]:
metrics = df1.iloc[:,7:15].columns
for i, metric in enumerate(metrics):
    #draw a boxplot
    g = sns.boxplot(y=metric,x="cultivar",data=df1, boxprops=dict(alpha=.3),order=order)
    g.set_xticklabels(g.get_xticklabels(),rotation=90,ha="center")
    
    #draw a swarmplot over the boxplot
    g = sns.swarmplot(y=metric,x="cultivar",data=df1,s=1.5,order=order)
    
    plt.tight_layout()
    plt.show()

## PCA

reuse the df1 variable above or read again with the below cell

In [ ]:
csv_path = "data/barley/extracted_phenotypes/area_morphology"

li = []
for i, file in enumerate([x for x in os.listdir(csv_path) if x.endswith(".csv")]):
    _df = pd.read_csv(os.path.join(csv_path,file), index_col=0)
    name = file.split(".")[0]
    _df.insert(loc=0,column="cultivar",value = name)
    _df.insert(loc=1,column="int_category",value = i)
    li.append(_df)

#create a dataframe from list
df1 = pd.concat(li, axis=0, ignore_index=True)

# retain only filtered values (i.e. removing non integral area such as awn containing, incomplete detection, partial appearance)
df1 = df1[df1["filter_level"] == 3]

# for easier interpretation of the dataframe
df1 = df1.sort_values('AS_seed_area')
df1.reset_index(drop=True, inplace=True)

In [ ]:
#normalize before PCA
_df1 = df1.iloc[:,7:15].apply(lambda x: (x-x.mean())/x.std(), axis=0)
#PCA
pca1 = PCA(n_components=None)
X = pca1.fit_transform(_df1)
embed1 = pd.DataFrame(X, columns=["PC{}".format(x + 1) for x in range(len(_df1.columns))])
embed1.index = _df1.index
embed1.insert(loc=0,column="cultivar",value = df1["cultivar"])
embed1

### Vanilla PCA Display

In [ ]:
#plot PCA on PC1 and PC2
pc = 1
g = sns.scatterplot(x="PC"+str(pc), y="PC"+str(pc+1),
                data=embed1, hue="cultivar", alpha=1,linewidth=0.1,s=10,hue_order=order)

#add variance explained to axis
plt.xlabel("PC%s (%s %%)" %
           (str(pc), np.round(pca1.explained_variance_ratio_[pc-1]*100, 2)))
plt.ylabel("PC%s (%s %%)" % (
    str(pc+1), np.round(pca1.explained_variance_ratio_[pc]*100, 2)))


#add legend
g.legend(loc="upper right",bbox_to_anchor=(1.5,1),ncol=2, borderaxespad=0.)

### Overlay Average Coordinates

In [ ]:
pc = 1
#calculate average of PCA coords
ave_embed1 = embed1.groupby("cultivar").mean()
ave_embed1["cultivar"] = ave_embed1.index


#plot PCA on PC1 and PC2
pc = 1
g = sns.scatterplot(x="PC"+str(pc), y="PC"+str(pc+1),
                data=embed1, hue="cultivar", alpha=1,linewidth=0.1,s=10,hue_order=order)

#plot average
g = sns.scatterplot(x="PC"+str(pc), y="PC"+str(pc+1),
                data=ave_embed1, alpha=1, edgecolor="black", linewidth=1,s=50,hue="cultivar",hue_order=order,legend=None)
#add cultivar name near average
for line in range(0,ave_embed1.shape[0]):
    textpos = [ave_embed1["PC"+str(pc)][line],ave_embed1["PC"+str(pc+1)][line]]
    plt.text(textpos[0], textpos[1],ave_embed1["cultivar"][line],size=12)


#add variance explained to axis
plt.xlabel("PC%s (%s %%)" %
           (str(pc), np.round(pca1.explained_variance_ratio_[pc-1]*100, 2)))
plt.ylabel("PC%s (%s %%)" % (
    str(pc+1), np.round(pca1.explained_variance_ratio_[pc]*100, 2)))
    
    
#add legend
g.legend(loc="upper right",bbox_to_anchor=(1.5,1),ncol=2, borderaxespad=0.)

### Overlay Eigenvector

In [ ]:
pc = 1

g = sns.scatterplot(x="PC"+str(pc), y="PC"+str(pc+1),
                data=embed1, hue="cultivar", alpha=1,linewidth=0.1,s=10,hue_order=order)
g.legend(loc="upper right",bbox_to_anchor=(1.5,1),ncol=2, borderaxespad=0.)

plt.xlabel("PC%s (%s %%)" %
           (str(pc), np.round(pca1.explained_variance_ratio_[pc-1]*100, 2)))
plt.ylabel("PC%s (%s %%)" % (
    str(pc+1), np.round(pca1.explained_variance_ratio_[pc]*100, 2)))
    

#eingenvector
xs = embed1.iloc[:, pc]
ys = embed1.iloc[:, pc+1]
coeff = 0.6 * np.transpose(pca1.components_[pc-1:pc-1+2, :])
n = coeff.shape[0]
scalex = 1.0/(xs.max() - xs.min())
scaley = 1.0/(ys.max() - ys.min())

for i, var in enumerate(list(_df1)):
    plt.arrow(0, 0, coeff[i, 0]/scalex, coeff[i, 1]/scaley, color="black")
    plt.text(coeff[i, 0]/scalex, coeff[i, 1]/scaley, var)

# Elliptic Fourier Descriptors

In [ ]:
# install via "pip install pyefd" if not installed
from pyefd import *

In [ ]:
#custom function to convert EFD from PC and other..
def pc2efv(pca, n, unit):
    vec = [0] * len(pca.explained_variance_)
    vec[n]= unit * np.sqrt(pca.explained_variance_[n])
    vec = np.dot(vec, pca.components_.T) + pca.mean_
    vvec = np.array([1,0,0])
    return np.concatenate([vvec,vec])
def pc2efm(res,n,unit):
    efm = pc2efv(res, n, -1*unit)
    efm = np.c_[efm,pc2efv(res,n,0)]
    efm = np.c_[efm,pc2efv(res,n,unit)]
    return efm.T

def get_value(pca,xi,yi):
    vec = [0] * len(pca.explained_variance_)
    vec[0] = xi
    vec[1] = yi
    vec = np.dot(vec, pca.components_.T) + pca.mean_
    vvec = np.array([1,0,0])
    return np.concatenate([vvec,vec])

In [ ]:
#import efd values

import glob
features = []
names = []
for file in glob.glob("data/barley/extracted_phenotypes/efd/*"):
    name = os.path.basename(file).split(".")[0]
    data = np.loadtxt(file,dtype=None,delimiter=",")
    names.extend([name]*len(data))
    features.extend(data)
#features = np.array(features)

fdf = pd.DataFrame(features).iloc[:,3:]
fdf.insert(loc=0,column="cultivar",value = names)
fdf.head()

## PCA

In [ ]:
#pca
pca2 = PCA(n_components=None)
X2 = pca2.fit_transform(fdf.iloc[:,1:])
embed2 = pd.DataFrame(X2, columns=["PC{}".format(x + 1) for x in range(len(fdf.columns[1:]))])
embed2.insert(loc=0,column="cultivar",value =names)
embed2.head()

In [ ]:
pc = 1
#calculate average of PCA coords
ave_embed2 = embed2.groupby("cultivar").mean()
ave_embed2["cultivar"] = ave_embed2.index


#plot PCA on PC1 and PC2
pc = 1
g = sns.scatterplot(x="PC"+str(pc), y="PC"+str(pc+1),
                data=embed2, hue="cultivar", alpha=1,linewidth=0.1,s=10,hue_order=order)

#plot average
g = sns.scatterplot(x="PC"+str(pc), y="PC"+str(pc+1),
                data=ave_embed2, alpha=1, edgecolor="black", linewidth=1,s=50,hue="cultivar",hue_order=order,legend=None)
#add cultivar name near average
for line in range(0,ave_embed2.shape[0]):
    textpos = [ave_embed2["PC"+str(pc)][line],ave_embed2["PC"+str(pc+1)][line]]
    plt.text(textpos[0], textpos[1],ave_embed2["cultivar"][line],size=8)

# the limits are too wide due to outliers. therefore manually setting it.
plt.xlim([-0.2,0.2])
plt.ylim([-0.04,0.04])
    
#add variance explained to axis
plt.xlabel("PC%s (%s %%)" %
           (str(pc), np.round(pca2.explained_variance_ratio_[pc-1]*100, 2)))
plt.ylabel("PC%s (%s %%)" % (
    str(pc+1), np.round(pca2.explained_variance_ratio_[pc]*100, 2)))
    
    
#add legend
g.legend(loc="upper right",bbox_to_anchor=(1.5,1),ncol=2, borderaxespad=0.)

## Latent Space Interpolation

In [ ]:
n = 8
grid_y = np.linspace(-0.03, 0.03, n)
grid_x = np.linspace(-0.25, 0.25, n)

k=1
for i, yi in enumerate(grid_y[::-1]):
    for j, xi in enumerate(grid_x):
        plt.subplot(n,n,k)
        plt.axis("off")
        value = get_value(pca2,xi,yi)
        contours = reconstruct_contour(value.reshape(20,4))
        plt.plot(contours[:,0],contours[:,1],color="black",linewidth=2)
        plt.fill_between(contours[:,0],contours[:,1],color="white",alpha=0.5)
        plt.xlim([-1.2,1.2])
        plt.ylim([-1,1])
        k +=1

## Visualize the shape correlated to each PCs

In [ ]:
plt.figure(figsize=(4,3))

for i in range(1,5):
    plt.subplot(2,2,i)
    plt.text(-0.2,-0.01,"PC"+str(i))
    plt.axis("off")
    n = i -1
    unit = 2
    efms = pc2efm(pca2, n, unit)
    for i, style in enumerate(["dashed","solid","dashed"]):
        contours = reconstruct_contour(efms[i].reshape(20,4))  #pyefd function
        plt.plot(contours[:,0],contours[:,1],linestyle=style,color="black")

    plt.tight_layout()

# Variational Autoencoder (VAE)

In [ ]:
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten, Lambda
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.losses import mse, binary_crossentropy
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
import keras.backend as K
from scipy.stats import norm

## Load model

In [ ]:
K.clear_session()

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon



batch_size = 1
image_size = 256
kernel_size = 3
filters = 32
latent_dim = 2
input_shape = (image_size, image_size, 3) 


inputs = Input(shape=input_shape, name='encoder_input')
x = inputs
for i in range(4):
    filters *= 2
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               activation='relu',
               strides=2,
               padding='same')(x)

shape = K.int_shape(x)
x = Flatten()(x)
x = Dense(16, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')


latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)
for i in range(4):
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        activation='relu',
                        strides=2,
                        padding='same')(x)
    filters //= 2

outputs = Conv2DTranspose(filters=3,
                          kernel_size=kernel_size,
                          activation='sigmoid',
                          padding='same',
                          name='decoder_output')(x)
decoder = Model(latent_inputs, outputs, name='decoder')


outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae') 

vae.compile(optimizer='rmsprop',loss="mse")  #a dummy loss. loss does not mean anything in the inference but needed for compiling.

#encoder.summary()
#decoder.summary()
vae.summary()
vae.load_weights("data/barley/model_weights/VAE.hdf5")

## Analyze latent vectors extracted from single seed images

In [ ]:
df3 = pd.read_csv("data/barley/extracted_phenotypes/VAE_latent.csv",index_col=0)
df3.head()

In [ ]:
#get averages as well
avedf3 = df3.groupby(["cultivar"]).mean()
avedf3= avedf3.rename(columns={0:"Z1",1:"Z2"})
avedf3.head()

## Visualize Latent Space

In [ ]:
plt.figure(figsize=(8,6))
g = sns.scatterplot(x="Z1",y="Z2",data=df3,hue = "cultivar",linewidth=0.1,alpha=1,s=10,legend=None,hue_order=order)

g = sns.scatterplot(x="Z1", y="Z2",
                data=avedf3, alpha=1, edgecolor="black", linewidth=1,s=50,hue=avedf3.index,hue_order=order)
g.legend(loc="upper left",bbox_to_anchor=(-0.2,-0.2),ncol=8, borderaxespad=0.)

for a, row in avedf3.iterrows():
    plt.text(row["Z1"],row["Z2"],a,size=8)
plt.tight_layout()

## Latent Space Interpolation

In [ ]:
#interpolation
n = 6
input_size = 256

xn = 8
yn = 8
grid_y = np.linspace(-8,5,yn) #これが実質y軸方向!
grid_x = np.linspace(-3,5,xn)

data = np.zeros(((input_size *yn), (input_size * xn), 3))
#grid_x = norm.ppf(np.linspace(0.01, 0.99, n))
#grid_y = norm.ppf(np.linspace(0.01, 0.99, n))

plt.figure(figsize=(8,7))
for i, yi in enumerate(grid_y[::-1]):
    for j, xi in enumerate(grid_x):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        image = x_decoded[0].reshape(input_size, input_size, 3)
        data[i * input_size: (i + 1) * input_size,
             j * input_size: (j + 1) * input_size] = image
plt.imshow(data)

#imsave("PCA_latent.jpg",data)